# DBSCAN

## Considerações

`eps` - Raio da circunferência à volta de um ponto que representa a sua vizinhança

`MinPts` - Número mínimo de pontos que queremos ver na vizinhança de um ponto

Ponto **core**
- Número de vizinhos $\ge$ MinPts

Ponto **boundary**
- Número de vizinhos < MinPts
- Deve estar na vizinhança de um ponto **core**

Ponto **noise**
- Não é ponto **core** nem **boundary**

Aresta de densidade - Une dois pontos **core** que sejam vizinhos

Pontos conectados por densidade - Diz-se que dois pontos **core** $p$ e $q$ estão conectados por densidade, caso estejam conectados através de arestas de densidade

## Algoritmo

1. Classificar os pontos como **core**, **boundary** ou **noise**

![Diagrama](files/dbscan1_core.png)

![Diagrama](files/dbscan1_boundary.png)

![Diagrama](files/dbscan1_noise.png)

2. Descartar os pontos **noise**

![Diagrama](files/dbscan2.png)

3. Atribuir um cluster a um ponto **core**

![Diagrama](files/dbscan3.png)

4. Colorir todos os pontos conectados por densidade de um ponto **core**

![Diagrama](files/dbscan4.png)

5. Repetir os passos 3 e 4 para os restantes pontos **core**

![Diagrama](files/dbscan5.png)

6. Colorir os pontos **boundary** de acordo com o ponto **core** mais próximo

![Diagrama](files/dbscan6.png)

## Implementação em Python

# KMeans

## Algoritmo

1. Escolher o número de clusters $k$

![Diagrama](files/kmeans1.png)

2. Selecionar $k$ pontos aleatórios como centróides

![Diagrama](files/kmeans2.png)

3. Atribuir cada ponto ao cluster cujo centróide se encontra mais próximo

![Diagrama](files/kmeans3.png)

4. Calcular a média de cada cluster como o novo centróide

![Diagrama](files/kmeans4.png)

5. Repetir passos 3 e 4

![Diagrama](files/kmeans5.png)

![Diagrama](files/kmeans6.png)